In [1]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from main import process_url


In [2]:
cross = "cross-validation.csv"
df = pd.read_csv(cross)
domains = df.loc[:, 'site'].to_list()
urls = df.loc[:, 'url'].to_list()
len(domains)

83

In [3]:
with ThreadPoolExecutor(max_workers=8) as executor:
    results = []
    for future in executor.map(process_url, domains):
        results.append(future)

In [16]:
for i, j in enumerate(zip(results, domains)):
    if j[0] is None:
        print(i, len(results), j)

18 83 (None, 'onstar.com')
72 83 (None, 'telus.net')
73 83 (None, 'nokia.com')
75 83 (None, 'greatcall.com')


In [17]:
cMatrix = {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}
FN = []
FP = []
for result, url, domain in zip(results, urls, domains):
    if result is not None:
        if result == url:
            cMatrix["TP"] += 1
        else:
            cMatrix["FP"] += 1
            FP.append((result, url, domain))
    else:
        if pd.isna(url):
            cMatrix["TN"] += 1
        else:
            cMatrix["FN"] += 1
            FN.append((result, url, domain))


In [27]:
Precision = cMatrix['TP'] / (cMatrix['TP'] + cMatrix['FP'])
Recall = cMatrix['TP'] / (cMatrix['TP'] + cMatrix['FN'])
F1 = 2 * ((Precision * Recall) / (Precision + Recall))
Accuracy = (cMatrix['TP'] + cMatrix['TN']) / (
    cMatrix['TP'] + cMatrix['FP'] + cMatrix['TN'] + cMatrix['FN'])
Specificity = cMatrix['TN'] / (cMatrix['TN'] + cMatrix['FP'])
NPV = cMatrix['TN'] / (cMatrix['TN'] + cMatrix['FN'])
print("Confusion matrix:", cMatrix, "\n")
print(f"{Precision=:.2f}")
print(f"{Recall=:.2f}")
print(f"{F1=:.2f}")
print(f"{Accuracy=:.2f}")
if cMatrix['TN'] != 0:
    print(f"{Specificity=:.2f} (lower is better)")
    print(f"{NPV=:.2f} (lower is better)")


Confusion matrix: {'TP': 65, 'FP': 14, 'TN': 3, 'FN': 1} 

Precision=0.82
Recall=0.98
F1=0.90
Accuracy=0.82
Specificity=0.18 (lower is better)
NPV=0.75 (lower is better)


In [28]:
for result, url, domain in FN:
    print("Got:", result)
    print("Cor:", url)
    print(domain, "FN")
    print()
for result, url, domain in FP:
    print("Got:", result)
    print("Cor:", url)
    print(domain, "FP")
    print()


Got: None
Cor: https://onstar.com/content/dam/onstar/na/us/en/navigation/01-images/os-header-logo-shared.svg
onstar.com FN

Got: https://superb.net/images2/main/4_accountrap-v1.png
Cor: https://superb.net/images2/main/logo.png
superb.net FP

Got: https://images.ctfassets.net/00i767ygo3tc/3nin3tyngzyx5yylb5hl8z/110981e92c9d86eae0a773356e6333e9/small-business-owner-using-a-free-video-maker-to-capture-attention-and-engage-audiences.webp?w=460&h=430&q=80
Cor: https://animoto.com/icons/icon-512x512.png?v=f7bad0df2a4af8688773dca5ee0b1ed6
animoto.com FP

Got: https://vanilla.futurecdn.net/cyclingnews/media/img/missing-image.svg
Cor: https://vanilla.futurecdn.net/toptenreviews/media/img/missing-image.svg
toptenreviews.com FP

Got: https://media-cldnry.s-nbcnews.com/image/upload/w_10000,h_74,q_80,c_fit,f_auto,q_auto:best/rockcms/2022-10/today-menu-2022-431ed9.png
Cor: https://cdn.cookielaw.org/logos/17e5cb00-ad90-47f5-a58d-77597d9d2c16/d44e374b-e570-4884-9441-33c0ccae5431/959d0f3c-d044-46db-bc4